In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## ARES XML Processing Functions & Common params

In [38]:
from lxml import etree as ET
import pandas as pd
import re

# input parameters
xml_namespaces = {'are': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_answer_or/v_1.0.2', 
              'dtt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_datatypes/v_1.0.2',
              'udt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/uvis_datatypes/v_1.0.1',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
             }
xml_data = ET.parse("darv_or.xml")

def strip_replace_new_line(tag_value):
    """
Remove /n from begining of tag value and end & replace with space when inside tag value
Args:
    tag_value: String tag value
Returns:
    tag_value w/o /n (string)
"""
    if tag_value:
        tag_value = tag_value.strip('\n')
        tag_value = tag_value.replace('\n',' ')
    return tag_value

def set_tag_value(attr_xpath, elem, xml_namespaces):
    """
Checks if tag/attribute @ exists and return its value
Args:
    attr_xpath: checked xpath
    elem: current element in xml
    xml_namespaces: namespaces in xml file
Returns:
    tag_value from xpath (string) 
"""
    if re.search('@',attr_xpath) is not None and elem.xpath(attr_xpath,namespaces = xml_namespaces) != []:# check for @ for attributes
        tag_value = elem.xpath(attr_xpath,namespaces = xml_namespaces)[0]
    elif elem.xpath(attr_xpath,namespaces = xml_namespaces) != []:
            tag_value = elem.xpath(attr_xpath,namespaces = xml_namespaces)[0].text
    else:
        tag_value = None
    return strip_replace_new_line(tag_value)

def populate_table (parent_tag, table_columns):
    """
Iterate elements under parent tag and get element tag/attr values for columns
Args:
    parent_tag: initial xml tag
    table_columns: xpaths to be checked and transformed to column values
Returns:
    table from xml data according to table_columns definition (pandas dataframe)
"""
    table_columns_pd = pd.DataFrame(table_columns,columns =['tag','xpath'])
    iter_tag_values = []
    for elem in parent_tag:
        tag_values = []
        for item in table_columns:
            tag_values.append(set_tag_value(item[1],elem, xml_namespaces)) 
        iter_tag_values.append(tag_values)
    table_pd = pd.DataFrame(iter_tag_values, columns=table_columns_pd['tag'].tolist())
    return table_pd

### zakladni_udaje

In [56]:
basic_info_table_columns = [
    # Vypis Vypis_OR/Uvod
    ['Aktualizace_DB','../dtt:Uvod/dtt:Aktualizace_DB'],
    ['Datum_vypisu','../dtt:Uvod/dtt:Datum_vypisu'],
    # Zakladni_udaje
    ['ICO','./dtt:ICO'],
    ['Stav_subjektu','./dtt:Stav/dtt:Stav_subjektu'],
    ['Konkurz','./dtt:Stav/dtt:Konkurz/dtt:Kod'],
    ['Vyrovnani','./dtt:Stav/dtt:Vyrovnani/dtt:Kod'],
    ['Zamitnuti','./dtt:Stav/dtt:Zamitnuti/dtt:Kod'],
    ['Likvidace','./dtt:Stav/dtt:Likvidace/dtt:Kod'],
    ['Datum_zapisu_OR','./dtt:Datum_zapisu_OR'],
    # Registrace
    
    ['Soud_Kod','../dtt:Registrace/dtt:Spisova_znacka/dtt:Soud/dtt:Kod'],
    ['Soud_Text','../dtt:Registrace/dtt:Spisova_znacka/dtt:Soud/dtt:Text'],
    ['Oddil_vlozka','../dtt:Registrace/dtt:Spisova_znacka/dtt:Oddil_vlozka'],
    
    # Pravni_forma_OR
    ['Kod_PF','./dtt:Pravni_forma_OR/dtt:Kod_PF'],
    ['Nazev_PF','./dtt:Pravni_forma_OR/dtt:Nazev_PF'],
    ['PF_osoba','./dtt:Pravni_forma_OR/dtt:PF_osoba'],
    ['TZU_osoba','./dtt:Pravni_forma_OR/dtt:TZU_osoba'],
    
]
basic_info_table_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje',namespaces = xml_namespaces)
populate_table(basic_info_table_parent_tag,basic_info_table_columns)

,Aktualizace_DB,Datum_vypisu,ICO,Stav_subjektu,Konkurz,Vyrovnani,Zamitnuti,Likvidace,Datum_zapisu_OR,Soud_Kod,Soud_Text,Oddil_vlozka,Kod_PF,Nazev_PF,PF_osoba,TZU_osoba
0,2018-08-22,2018-08-23,63275635,Aktivní,0,0,0,0,1995-08-01,3,Krajský soud v Českých Budějovicích,B 698,121,Akciová společnost,P,Tuzemská


### obchodni_firma

In [46]:
company_name_table_columns = [
    ['ICO','../dtt:ICO'],
    ['dod','./@dod'],
    ['ddo','./@ddo'],
    ['Obchodni_firma','.']
]
company_name_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Obchodni_firma',namespaces = xml_namespaces)
populate_table(company_name_parent_tag,company_name_table_columns)

,ICO,dod,ddo,Obchodni_firma
0,63275635,2006-06-30,None,MADETA a. s.
1,63275635,2005-07-04,2006-06-30,MADETA Group a. s.
2,63275635,1995-08-01,2005-07-04,"JIHOMILK, akciová společnost"


### sidlo

In [48]:
headquarters_table_columns = [
    ['ICO','../dtt:ICO'],
    ['dod','./@dod'],
    ['ddo','./@ddo'],
    ['ID_adresy','./dtt:ID_adresy'],
    ['Kod_statu','./dtt:Kod_statu'],
    ['Nazev_statu','./dtt:Nazev_statu'],
    ['Nazev_okresu','./dtt:Nazev_okresu'],
    ['Nazev_obce','./dtt:Nazev_obce'],
    ['Nazev_casti_obce','./dtt:Nazev_casti_obce'],
    ['Nazev_ulice','./dtt:Nazev_ulice'],
    ['Cislo_domovni','./dtt:Cislo_domovni'],
    ['Typ_cislo_domovni','./dtt:Typ_cislo_domovni'],
    ['Cislo_orientacni','./dtt:Cislo_orientacni'],
    ['PSC','./dtt:PSC'],
    ['Cislo_do_adresy','./dtt:Cislo_do_adresy']
                      ]
headquarters_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Sidlo',namespaces = xml_namespaces)
populate_table(headquarters_parent_tag,headquarters_table_columns)

,ICO,dod,ddo,ID_adresy,Kod_statu,Nazev_statu,Nazev_okresu,Nazev_obce,Nazev_casti_obce,Nazev_ulice,Cislo_domovni,Typ_cislo_domovni,Cislo_orientacni,PSC,Cislo_do_adresy
0,63275635,2017-11-23,None,29643147,203,Česká republika,České Budějovice,České Budějovice,České Budějovice 4,Rudolfovská tř.,246,1,83,37001,None
1,63275635,2016-12-07,2017-11-23,29643145,203,Česká republika,České Budějovice,České Budějovice,České Budějovice 4,Rudolfovská tř.,246,1,83,37001,None
2,63275635,2002-08-14,2016-12-07,29643143,203,Česká republika,České Budějovice,České Budějovice,None,Rudolfovská,None,None,None,37050,246/83
3,63275635,1995-08-01,2002-08-14,29643141,203,Česká republika,None,České Budějovice,None,Rudolfovská,None,None,None,37050,83


### predmet_podnikani
__TODO__: 

Rozmyslet jestli brat i ostatni cinnosti (ucel nadace atp.)

In [50]:
cinnosti_table_columns = [
    ['ICO','../../../dtt:Zakladni_udaje/dtt:ICO'],
    ['xx','./are:Odpoved/dtt:Vypis_OR/dtt:Cinnosti/dtt:Predmet_podnikani/dtt:Text'],
    ['dod','./@dod'],
    ['ddo','./@ddo'],
    ['predmet_podnikani','.']
]
cinnosti_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Cinnosti/dtt:Predmet_podnikani/dtt:Text',namespaces = xml_namespaces)
populate_table(cinnosti_parent_tag,cinnosti_table_columns)

,ICO,xx,dod,ddo,predmet_podnikani
0,63275635,None,2014-08-06,None,"prodej kvasného lihu, konzumního lihu a lihovin"
1,63275635,None,2011-11-10,None,opravy silničních vozidel
2,63275635,None,2011-11-10,None,klempířství a oprava karoserií
3,63275635,None,2009-03-06,None,"zámečnictví, nástrojářství"
4,63275635,None,2009-03-06,None,"výroba, obchod a služby neuvedené v přílohách ..."
5,63275635,None,2009-03-06,None,"výroba, instalace, opravy elektrických strojů ..."
6,63275635,None,2008-01-22,2009-03-06,"testování, měření, analýzy a kontroly"
7,63275635,None,2007-10-11,2015-04-18,koupě za účelem jeho dalšího prodeje a prodej
8,63275635,None,2007-10-11,2015-04-18,velkoobchod
9,63275635,None,2007-10-11,2015-04-18,zprostředkovatelská činnost


### statutarni_organ
__TODO:__ zkontrolovat v profilingu, existenci FO bez bydliste v takovem pripade jeste iterovat pres FO a nasledne spojit DFs pres unique

In [58]:
statutory_table_columns = [
    ['ICO','../../dtt:Zakladni_udaje/dtt:ICO'],
    ['dod','./@dod'],
    ['ddo','./@ddo'],
    ['Kod_angm','./dtt:Clen/dtt:Kod_angm'],
    ['Funkce','./dtt:Clen/dtt:Funkce'],
    ['Titul_pred','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Titul_pred'],
    ['Jmeno','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Jmeno'],
    ['Prijmeni','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Prijmeni'],
    ['Titul_za','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Titul_za'],
    ['Datum_narozeni','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Datum_narozeni'],
    ['Rodne_cislo','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Rodne_cislo'],
    ['Osoba_textem','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Osoba_textem'],
    ['Bydliste_dod','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/@dod'],
    ['Bydliste_ddo','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/@ddo'],
    ['ID_adresy','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:ID_adresy'],
    ['Kod_statu','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Kod_statu'],
    ['Nazev_statu','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Nazev_statu'],
    ['Nazev_okresu','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Nazev_okresu'],
    ['Nazev_obce','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Nazev_obce'],
    ['Nazev_casti_obce','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Nazev_casti_obce'],
    ['Nazev_ulice','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Nazev_ulice'],
    ['Cislo_domovni','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Cislo_domovni'],
    ['Typ_cislo_domovni','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Typ_cislo_domovni'],
    ['Cislo_orientacni','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Cislo_orientacni'],
    ['PSC','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:PSC'],
    ['Cislo_do_adresy','./dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste/dtt:Cislo_do_adresy']
]
statutory_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Statutarni_organ/dtt:Clen_SO',namespaces = xml_namespaces)
data = populate_table(statutory_parent_tag,statutory_table_columns)
data.head(100)

,ICO,dod,ddo,Kod_angm,Funkce,Titul_pred,Jmeno,Prijmeni,Titul_za,Datum_narozeni,...,Nazev_statu,Nazev_okresu,Nazev_obce,Nazev_casti_obce,Nazev_ulice,Cislo_domovni,Typ_cislo_domovni,Cislo_orientacni,PSC,Cislo_do_adresy
0,63275635,2017-09-13,None,1,místopředseda představenstva,Ing.,PAVEL,Tester,None,1984-12-02,...,None,None,None,None,None,None,None,None,None,None
1,63275635,2017-09-13,None,1,místopředseda představenstva,Ing.,PAVEL,BURYÁNEK,None,1984-12-02,...,Česká republika,Tábor,Veselí nad Lužnicí,Veselí nad Lužnicí II,Na Pasekách,948,1,None,39181,None
2,63275635,2016-12-07,None,1,předseda představenstva,Ing.,MILAN,TEPLÝ,None,1949-03-26,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
3,63275635,2016-12-07,None,1,člen představenstva,Ing.,PETR,PAYER,None,1976-07-17,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 3,Nerudova,904,1,66,37004,None
4,63275635,2016-12-07,None,1,člen představenstva,Ing.,JAN,TEPLÝ,None,1980-03-15,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
5,63275635,2016-12-07,2017-09-13,1,místopředseda představenstva,Ing.,PAVEL,BURYÁNEK,None,1984-12-02,...,Česká republika,Tábor,Veselí nad Lužnicí,Veselí nad Lužnicí I,A. Jiráska,349,1,None,39181,None
6,63275635,2014-08-06,2016-12-07,1,místopředseda představenstva,Ing.,PAVEL,BURYÁNEK,None,1984-12-02,...,Česká republika,None,Veselí nad Lužnicí,Veselí nad Lužnicí I,A. Jiráska,349,1,None,39181,None
7,63275635,2014-08-06,2016-12-07,1,člen představenstva,Ing.,PETR,PAYER,None,1976-07-17,...,Česká republika,None,České Budějovice,České Budějovice 3,Nerudova,904,1,66,37004,None
8,63275635,2014-08-06,2016-12-07,1,člen představenstva,Ing.,JAN,TEPLÝ,None,1980-03-15,...,Česká republika,None,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
9,63275635,2013-12-19,2014-08-06,1,člen představenstva,Ing.,JAN,TEPLÝ,None,1980-03-15,...,Česká republika,None,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
